In [ ]:
This notebook performs EDA on `safe_data.csv`, prepares sliding windows, trains a 1D-CNN autoencoder on *safe* data, and demonstrates anomaly detection with synthetic anomalies.</VSCode.Cell>
<VSCode.Cell language="python"># 2. Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.utils import load_csv, compute_magnitude
from src.dataset import windows_from_array
%matplotlib inline</VSCode.Cell>
<VSCode.Cell language="python"># 3. Load data and EDA
df = load_csv('safe_data.csv')
print(df.shape)
print(df.head())
print(df.describe())

df['mag'] = compute_magnitude(df)
plt.figure(figsize=(10,6))
plt.plot(df['AccX'], label='AccX')
plt.plot(df['AccY'], label='AccY')
plt.plot(df['AccZ'], label='AccZ')
plt.legend()
plt.title('Accelerometer axes (index as time)')
plt.show()

plt.figure(figsize=(10,2))
plt.plot(df['mag'], label='magnitude')
plt.title('Magnitude')
plt.show()</VSCode.Cell>
<VSCode.Cell language="python"># 4. Create windows and visualize one window
arr = df[['AccX','AccY','AccZ']].values
window = 128
stride = 64
wins = windows_from_array(arr, window=window, stride=stride)
print('windows:', wins.shape)

plt.figure(figsize=(8,4))
plt.plot(wins[10,:,:])
plt.title('Window sample (channels along columns)')
plt.show()</VSCode.Cell>
<VSCode.Cell language="python"># 5. Quick small training run (smoke test)
# Use the provided training script with small epoch to validate
!python -m src.train --data safe_data.csv --outdir checkpoints_test --window 64 --stride 32 --epochs 3 --batch_size 16
</VSCode.Cell>
<VSCode.Cell language="python"># 6. Load model and show reconstruction errors
from src.inference import load_model_and_meta, classify_series
model, scaler, thresh = load_model_and_meta('checkpoints_test')
arr = df[['AccX','AccY','AccZ']].values
is_win, errs, sample_flags = classify_series(arr, model, scaler, thresh, window=64, stride=32)

plt.figure(figsize=(10,3))
plt.plot(errs)
plt.axhline(thresh, color='red', linestyle='--', label='threshold')
plt.legend()
plt.title('Per-window reconstruction errors')
plt.show()

print('fraction anomalous windows:', is_win.mean())
</VSCode.Cell>
<VSCode.Cell language="python"># 7. Synthetic anomaly test: inject spikes and shifts
arr2 = arr.copy()
# inject spike at sample index 150
arr2[150:160] += np.array([50.0, -30.0, 10.0])
# inject sustained offset from 400:450
arr2[400:450] += np.array([5.0,5.0,5.0])
is_win2, errs2, sample_flags2 = classify_series(arr2, model, scaler, thresh, window=64, stride=32)

plt.figure(figsize=(10,3))
plt.plot(errs2)
plt.axhline(thresh, color='red', linestyle='--')
plt.title('Errors with synthetic anomalies')
plt.show()

plt.figure(figsize=(10,2))
plt.plot(sample_flags2.astype(int))
plt.title('Sample-level anomaly flags (1 means flagged)')
plt.show()</VSCode.Cell>
<VSCode.Cell language="markdown">## Next steps
- Run a longer training with larger window (128) and more epochs, choose threshold carefully.
- Add synthetic anomaly generation strategies (spikes, drifts, noise, sensor dropout) and evaluate detection metrics.
- Optionally, create a FastAPI inference endpoint and dockerize the app.